# DISCOV-Gracilaria Paper

Simon Oiry¹ [](https://orcid.org/0000-0001-7161-5246) (, Institut des Substances et Organismes de la Mer, ISOMer, , , )  
Bede Ffinian Rowe Davies¹ [](https://orcid.org/0000-0001-6462-4347) ()  
Pierre Gernez¹ [](https://orcid.org/0000-0003-2055-410X) ()  
Laurent Barillé¹ [](https://orcid.org/0000-0001-5138-2684) ()  
November 20, 2024

To be Written

[1]

[1] Institut des Substances et Organismes de la Mer, ISOMer, Nantes Université, UR 2160, F-44000 Nantes, France

In [ ]:
library(tidyverse)
library(terra)
library(rnaturalearth) 
library(rnaturalearthdata) 
library(sf)
library(Utilities.Package)
library(flextable)


Attachement du package : 'flextable'

Les objets suivants sont masqués depuis 'package:terra':

    align, colorize, rotate, width

L'objet suivant est masqué depuis 'package:purrr':

    compose

In [ ]:
my_comma<-scales::label_comma(accuracy = NULL, big.mark = ",",decimal.mark = ".")

# 1. Introduction

Coastal ecosystems are among the most dynamic and productive environments on Earth, providing ecosystem services and supporting biodiversity ([Barbier et al., 2011](#ref-barbier2011value); [Unsworth et al., 2022](#ref-unsworth2022planetary); [Watanabe et al., 2018](#ref-watanabe2018introduction)). These ecosystems, spanning mangroves, salt marshes, seagrass meadows, and rocky intertidal zones, play a pivotal role in carbon sequestration, nutrient cycling, and shoreline stabilization ([Liquete et al., 2013](#ref-liquete2013current); [Mehvar et al., 2018](#ref-mehvar2018quantifying)). They also serve as habitats for numerous species, many of which are commercially or ecologically significant ([De Valck et al., 2023](#ref-de2023valuing); [Seitz et al., 2014](#ref-seitz2014ecological)). Coastal areas are densely populated, with billions of people globally depending on their resources for livelihoods, fisheries, and tourism ([Mukherjee et al., 2023](#ref-mukherjee2023coastal); [Small and Nicholls, 2003](#ref-small2003global)). However, coastal ecosystems face mounting pressures from human activities such as land reclamation, pollution, and overfishing, compounded by the impacts of climate change ([Hall-Spencer and Harvey, 2019](#ref-hall2019ocean); [Lu et al., 2018](#ref-lu2018major)). Sea level rise, ocean acidification, and increasing storm intensity further exacerbate the vulnerability of these systems, threatening their resilience and the services they provide ([He and Silliman, 2019](#ref-he2019climate)).

One of the significant threats to coastal ecosystems is biological invasions by non-native species, which can disrupt native biodiversity and alter ecosystem functions ([Capdevila et al., 2019](#ref-capdevila2019warming); [Krueger-Hadfield, 2018](#ref-krueger2018everywhere); [Liu et al., 2020](#ref-liu2020ocean)). *Gracilaria vermiculophylla*, an invasive red macroalga native to the northwest Pacific, exemplifies this issue. Over the last century, this species has spread extensively across temperate estuaries in North America, Europe, and other regions, facilitated by aquaculture and maritime activities ([Krueger-Hadfield et al., 2017](#ref-krueger2017genetic); [Rueness, 2005](#ref-rueness2005life); [Weinberger et al., 2008](#ref-weinberger2008invasive)). Its success as an invader stems from its tolerance to a wide range of environmental stressors, including temperature ([Sotka et al., 2018](#ref-sotka2018combining)), salinity ([Weinberger et al., 2008](#ref-weinberger2008invasive)), and nutrient variability ([Abreu et al., 2011](#ref-abreu2011nitrogen)), as well as its ability to establish in soft sediment habitats traditionally devoid of macroalgae ([Ramus et al., 2017](#ref-ramus2017invasive)). While *G. vermiculophylla* can provide some ecosystem services, such as habitat for invertebrates and juvenile fish, it often outcompetes native vegetation, alters sediment composition ([Nyberg et al., 2009](#ref-nyberg2009flora)), and disrupts trophic interactions ([Ginneken et al., 2018](#ref-van2018global)). In regions like the Baltic Sea and the eastern United States, it has been documented to negatively affect native fucoids and seagrasses ([Thomsen et al., 2013](#ref-thomsen2013effects); [Van Katwijk, 2003](#ref-van2003reintroduction)). These impacts underscore the importance of monitoring and managing the spread of *G. vermiculophylla*, particularly as climate change and anthropogenic pressures continue to facilitate biological invasions.

Remote sensing has revolutionized our ability to monitor and manage ecosystems, offering efficient and scalable methods for detecting environmental changes over large areas ([Davies et al., 2024a](#ref-davies2024intertidal), [2024b](#ref-davies2024sentinel); [Zoffoli et al., 2021](#ref-zoffoli2021decadal)). Among these technologies, drone-based remote sensing has emerged as a particularly promising tool for studying coastal environments ([Román et al., 2024](#ref-roman2024mapping), [2023](#ref-roman2023mapping)). Equipped with high-resolution cameras and multispectral or hyperspectral sensors, drones can capture fine-scale spatial and spectral data, enabling researchers to identify and map vegetation, detect stress in plants, and monitor changes over time \[Román et al. ([2021](#ref-roman2021using)) ; Oiry et al. 2024\]. Unlike traditional satellite imagery, drones provide the flexibility to operate in overcast conditions, achieve higher spatial resolution, and target specific areas of interest. For invasive species like *G. vermiculophylla*, drones equipped with multispectral sensors can differentiate it from native vegetation based on its unique spectral reflectance characteristics (Davies et al. 2025). This capability not only enhances detection accuracy but also reduces the time and labor associated with traditional field surveys. As the cost of drone technology continues to decrease and advancements in machine learning facilitate data analysis, drone-based remote sensing is becoming increasingly accessible and impactful for ecological research and management.

In this study, we aim to harness the potential of drone-based multispectral remote sensing to map *Gracilaria vermiculophylla* in intertidal zones. Bla bla what are we going to do ? bla bla .

# 2. Materiel & Methods

## 2.1 Study sites

Field campaigns were conducted at three study sites in France and Spain. At each site, two locations were investigated <a href="#fig-location_sites" class="quarto-xref">Figure 1</a>. The Aven & Belon Estuary in South Brittany, France (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a> A), is a dynamic ria-type system hosting diverse habitats, including sandy tidal flats and subtidal zones with coarse, marine-origin sediments ([Castaing and Guilcher, 1995](#ref-Castaing1995); [Michel et al., 2021](#ref-Michel2021)). These habitats support key benthic species such as *Scrobicularia plana*, *Cerastoderma edule*, and *Tellina tenuis*, which play essential roles in sediment bioturbation and nutrient cycling ([Blanchet et al., 2014](#ref-Blanchet2014); [Tankoua et al., 2011](#ref-Tankoua2011)). The estuary serves as a nursery for juvenile fish and a feeding ground for migratory birds, with its ecological productivity driven by a mix of euryhaline and marine species adapted to salinity gradients ([Blanchet et al., 2014](#ref-Blanchet2014)). Oyster farming, particularly *Crassostrea gigas*, is a dominant activity, altering sediment dynamics and local biodiversity ([Michel et al., 2021](#ref-Michel2021)). Despite its ecological richness, the estuary faces pressures from nutrient loading and physical alterations, with bioindicators like *S. plana* used to monitor the impacts of salinity, sediment quality, and pollution ([Tankoua et al., 2011](#ref-Tankoua2011)).

The Ria d’Étel, located in Brittany, France, is a macrotidal estuary characterized by its unique hydrodynamics and biodiversity (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a> B). Influenced predominantly by tidal regimes, the estuary exhibits high-energy zones with strong currents reaching up to 2.5 m/s, shaping both sediment deposition and ecological habitats ([Portas et al., 2023](#ref-Portas2023)). The estuary supports diverse benthic communities, with sedimentary organic matter originating from both terrestrial inputs and marine sources, contributing to nutrient cycling and benthic fluxes ([Jeanneau et al., 2023](#ref-Jeanneau2023)). Vegetation gradients transition from halophytic plants in saline zones to freshwater species upstream, reflecting the estuary’s salinity dynamics and ecological complexity ([Cianfaglione, 2021](#ref-Cianfaglione2018)). This estuary is also notable for its shellfish farming, with species like *Crassostrea gigas* cultivated extensively. The presence of filter-feeding organisms such as sponges (*Hymeniacidon perlevis*) enhances water quality by mitigating bacterial loads and promoting bioremediation ([Gentric and Sauleau, 2024](#ref-Gentric2024)). However, the estuary faces environmental pressures, including nutrient enrichment from agricultural runoff and anthropogenic impacts on sedimentary processes.

The Saja-Besaya Estuary, situated along the Cantabrian Sea in northern Spain, is characterized by the confluence of the Saja and Besaya rivers near Torrelavega (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a> C). The estuary, also known as San Martín de la Arena or Suances Estuary, has been subject to significant anthropogenic pressures, including industrial developments throughout the 20th century. These activities have led to contamination from mining, paper manufacturing, and carbonate discharges, classifying the estuary as highly polluted near its upper reaches ([Ortega et al., 2005](#ref-ortega2005fluxes)). This contamination impacts the estuarine ecosystem, including water quality and biodiversity, with minimal aquatic life and sparse riverbank vegetation in its lower sections ([Romero et al., 2008](#ref-romero2008sintering)).

In [ ]:
######### Aven Belon 

mask_aven_belon <- read_sf("Data/shp/mask/mask_site_Aven_Belon_32630.shp")

HT <-  list.files("Data/Sentinel2/Aven_Belon/S2A_MSIL2A_20231107T111251_N0509_R137_T30TVT_20231107T144858.SAFE/GRANULE/L2A_T30TVT_A043749_20231107T111246/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_aven_belon, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_aven_belon, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)


LT <-  list.files("Data/Sentinel2/Aven_Belon/S2A_MSIL2A_20240624T110641_N0510_R137_T30TVT_20240624T153247.SAFE/GRANULE/L2A_T30TVT_A047038_20240624T111001/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_aven_belon, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_aven_belon, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Aven_Belon <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Aven_Belon)[values(intertidal_Aven_Belon) == F] <- NA

mask_sf <- as.polygons(intertidal_Aven_Belon) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Aven_Belon.shp")


Land_Aven_Belon <- NDVI_LT > 0.05 & NDVI_HT > 0.05


values(Land_Aven_Belon)[values(Land_Aven_Belon) == F] <- NA

mask_sf <- as.polygons(Land_Aven_Belon) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes()
  # smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Aven_Belon.shp")



######### ETEL

mask_Etel <- read_sf("Data/shp/mask/mask_site_Etel_32630.shp")

HT <-  list.files("Data/Sentinel2/Etel/S2A_MSIL2A_20231008T110941_N0509_R137_T30TVT_20231008T171550.SAFE/GRANULE/L2A_T30TVT_A043320_20231008T111641/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_Etel, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_Etel, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)

LT <-  list.files("Data/Sentinel2/Etel/S2B_MSIL2A_20230718T112119_N0509_R037_T30TVT_20230718T124824.SAFE/GRANULE/L2A_T30TVT_A033239_20230718T112118/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_Etel, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_Etel, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Etel <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Etel)[values(intertidal_Etel) == F] <- NA

mask_sf <- as.polygons(intertidal_Etel) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Etel.shp")


Land_Etel <- NDVI_LT > 0.05 & NDVI_HT > 0.05


values(Land_Etel)[values(Land_Etel) == F] <- NA

mask_sf <- as.polygons(Land_Etel) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() 
  
plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Etel.shp")

######### SAJA

mask_SAJA <- read_sf("Data/shp/mask/mask_site_Saja_32630.shp")


HT <-  list.files("Data/Sentinel2/Saja/S2A_MSIL2A_20181004T110911_N0500_R137_T30TVP_20230620T014212.SAFE/GRANULE/L2A_T30TVP_A017151_20181004T111207/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_SAJA, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_SAJA, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)


LT <-  list.files("Data/Sentinel2/Saja/S2A_MSIL2A_20230928T110831_N0509_R137_T30TVP_20230928T171657.SAFE/GRANULE/L2A_T30TVP_A043177_20230928T111532/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_SAJA, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_SAJA, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Saja <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Saja)[values(intertidal_Saja) == F] <- NA

mask_sf <- as.polygons(intertidal_Saja) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Saja.shp")


Land_Saja <- NDVI_LT > 0.05 & NDVI_HT > 0.05

values(Land_Saja)[values(Land_Saja) == F] <- NA

mask_sf <- as.polygons(Land_Saja) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Saja.shp")

In [ ]:
sovereignty10 <- ne_countries(scale = 10, returnclass = "sf")

world_map <- sovereignty10 %>% 
  st_as_sf() %>% 
  dplyr::filter(sovereignt%in%c("Spain","France","Portugal",
                                "Italy","Andorra",
                                "United Kingdom",
                                "Switzerland","Belgium",
                                "Germany","Luxembourg") ) 

bbox_europe <- st_bbox(c(xmin = -20, ymin = 34,
                         xmax = 20, ymax = 55) ,
                       crs = st_crs(world_map) ) 

world_map<-st_make_valid(world_map) 

european_union_map_cropped <- st_crop(world_map, bbox_europe)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  


UnitedKingdom<-sovereignty10 %>% 
  st_as_sf() %>% 
  dplyr::filter(sovereignt%in%c("United Kingdom") ) %>% 
  st_cast("POLYGON") 

bbox_UK <- st_bbox(c(xmin = -20, ymin = 45,
                         xmax = 20, ymax = 55) ,
                       crs = st_crs(UnitedKingdom) ) 

UnitedKingdom<-st_make_valid(UnitedKingdom) 

UK_map_cropped <- st_crop(UnitedKingdom, bbox_UK)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ") 

Europe_sf<-european_union_map_cropped %>% 
  dplyr::bind_rows(UK_map_cropped) 


Projects<-data.frame(
  Name=c(
  "France -Ria d'Etel",
  "France - Aven & Belon",
  "Spain - Saja Estuary"
  ) ,
  Long=c(-3.187818,
         -3.724905, 
         -4.025402) ,
  Lat=c(47.697566, 
        47.824779,
        43.408356) 
  )  %>% 
  st_as_sf(coords=c("Long","Lat") )  %>% 
  st_set_crs("EPSG:4326")  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  

Projects_df<-Projects %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>% 
  sf::st_set_geometry(NULL)

Miniworld_map <- sovereignty10 %>% 
  st_as_sf()

sf_use_s2(FALSE)

bbox_EU <- st_bbox(c(xmin = -30, ymin = 20,
                         xmax = 50, ymax = 70) ,
                       crs = st_crs(Miniworld_map) ) 

  MiniEU_map<-st_crop(Miniworld_map, bbox_EU)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  

  df_P1<-Projects_df %>% 
    mutate(ID = c(1:nrow(.))) %>% 
    mutate(Site = case_when(ID == 1 ~ "Ria d'Etel",
                            ID == 2 ~ "Aven & Belon",
                            ID == 3 ~ "Saja Estuary",
                            TRUE ~ "NA"),
           ID = case_when(ID == 1 ~ "B",
                          ID == 2 ~ "A",
                          TRUE ~ "C"),
           lon_label = 2900000,
           lat_label = case_when(Site == "Ria d'Etel" ~ lat  - 200000,
                                 Site == "Aven & Belon" ~ lat + 50000,
                                 Site == "Saja Estuary" ~ lat  - 250000,),
           lat_ID = lat_label + 100000) %>% 
    dplyr::select(c(lon,lat,lon_label,lat_label,lat_ID,ID,Site))
    
    scaleFUN <- function(x) paste0(sprintf("%.2f", x),"°N")
(
p1 <- ggplot(MiniEU_map) +
  geom_sf(linewidth = 0.5, alpha = 0.93, fill = "#CFCFCF", colour = "grey30") +
  geom_segment(data = df_P1, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = ID), color = "goldenrod4",size = 1)+
  geom_point(data = df_P1, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = df_P1, aes(x = lon_label, y =lat_label, label = Site), fill = "#FFD27D",size = 5)+
  geom_label(data = df_P1, aes(x = lon_label, y = lat_ID, label = c("B","A","C")), size = 6)+
  coord_sf(
    xlim = c(2600000, 4100000),
    ylim = c(1600000, 3100000), 
    expand = FALSE
  ) +
  theme_Bede_Map() +
  labs(x = "Longitude", y = "Latitude") +
  scale_x_continuous(breaks = 0) +
  scale_y_continuous(breaks = 42, labels = scaleFUN) +
  # theme_void() +
  theme(
    plot.margin = unit(c(0, 0, 0, 0), "cm"),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    axis.text.x = element_text(size = 20, vjust = 7.1, hjust = 0.8),
    axis.text.y = element_text(size = 20, vjust = -5, hjust = 0, angle = 90),
    # panel.border = element_rect(color = "black", fill = NA),
    # panel.background = element_rect(fill = "white"),
    # plot.background = element_rect(fill = "white", color = NA),
    # legend.background = element_rect(fill = "transparent"),
    # panel.grid.major = element_line(color = "grey", linetype = "dashed", linewidth = 0.5),
    # panel.grid.minor = element_line(color = "grey", linetype = "dotted", linewidth = 0.25)
  ))



ggsave("Paper/Figures/High_res/Figure1/Map_Drone_Sites.png",p1,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Drone_Sites.png",p1,width= 10, height=10, dpi = 200) 

In [ ]:
create_square_sf <- function(side_length, center_lonlat) {
  # Validate input
  if (length(center_lonlat) != 2) {
    stop("Please provide the center coordinates as a vector: c(longitude, latitude).")
  }
  if (!is.numeric(side_length) || side_length <= 0) {
    stop("Please provide a positive numeric value for the side length.")
  }
  
  # Define the center point in lon/lat
  center_point <- st_sfc(st_point(center_lonlat), crs = 4326)
  
  # Transform to EPSG:32630
  center_point_utm <- st_transform(center_point, 32630)
  center_coords <- st_coordinates(center_point_utm)
  
  # Calculate half side length in meters
  half_side <- side_length / 2
  
  # Create the square corners in UTM
  square_coords <- matrix(c(
    center_coords[1] - half_side, center_coords[2] - half_side,
    center_coords[1] + half_side, center_coords[2] - half_side,
    center_coords[1] + half_side, center_coords[2] + half_side,
    center_coords[1] - half_side, center_coords[2] + half_side,
    center_coords[1] - half_side, center_coords[2] - half_side # Close the square
  ), ncol = 2, byrow = TRUE)
  
  # Create a polygon from the coordinates
  square_polygon <- st_polygon(list(square_coords))
  
  # Create an sf object
  square_sf <- st_sfc(square_polygon, crs = 32630)
  
  return(square_sf)
}

In [ ]:
It_Saja <- "Data/shp/Intertidal_sites/mask_intertidal_Saja.shp" %>% 
  read_sf()

Land_Saja <- "Data/shp/Intertidal_sites/mask_Land_Saja.shp" %>% 
  read_sf()

ext_plot_Saja <- st_bbox(create_square_sf(5000, c(-4.025987, 43.4200065)))



Flight_Saja<-data.frame(
  Name=c(
  "Marisma de \nCortiguera",
  "Marisma de \nCudón"),
  Long=c(-4.029591,
         -4.031800) ,
  Lat=c(43.409738,
        43.414434)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Marisma de \nCudón" ~ 418639.3,
                               Name == "Marisma de \nCortiguera" ~ lon + 2000),
         lat_label = case_when(Name == "Marisma de \nCudón" ~ lat + 1000,
                               Name == "Marisma de \nCortiguera" ~ lat + 500))

(plot_Saja <- ggplot() +
  geom_sf(data = Land_Saja, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_Saja, linewidth = 0.05, alpha = 0.93, fill = "goldenrod")+
    geom_label(aes(x = as.numeric(ext_plot_Saja[1]) + (as.numeric(ext_plot_Saja[3])-as.numeric(ext_plot_Saja[1]))*0.1 , 
                   y = as.numeric(ext_plot_Saja[2]) + (as.numeric(ext_plot_Saja[4])-as.numeric(ext_plot_Saja[2]))*0.1
                   , label = "C"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_Saja[1], ext_plot_Saja[3]),
    ylim = c(ext_plot_Saja[2], ext_plot_Saja[4]),
    expand = F
  ) +
  geom_segment(data = Flight_Saja, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_Saja, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_Saja, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map()+
  labs(x="Longitude",
       y="Latitude")+
  scale_x_continuous(breaks = -4.01) +
  scale_y_continuous(breaks = 43.415) +
  # Move y-axis to the right
  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 4.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -4, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank()
    )  # Adjust position if needed  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_Saja.png",plot_Saja,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Saja.png",plot_Saja,width= 10, height=10, dpi = 200)  

In [ ]:
It_AB <- "Data/shp/Intertidal_sites/mask_intertidal_Aven_Belon.shp" %>% 
  read_sf()

Land_AB <- "Data/shp/Intertidal_sites/mask_Land_Aven_Belon.shp" %>% 
  read_sf()



ext_plot_AB <- st_bbox(create_square_sf(10000, c(-3.706226, 47.815149)))


Flight_AB<-data.frame(
  Name=c(
  "Pont du Guilly",
  "Notre-Dame De Tremor"),
  Long=c(-3.655220,
         -3.748634) ,
  Lat=c(47.822408,
        47.837923)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Pont du Guilly" ~ lon - 1000,
                               Name == "Notre-Dame De Tremor" ~ lon + 2000),
         lat_label = case_when(Name ==  "Pont du Guilly" ~ lat - 1000,
                               Name == "Notre-Dame De Tremor" ~ lat + 1500))

(plot_AB <- ggplot() +
  geom_sf(data = Land_AB, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_AB, linewidth = 0.05, alpha = 0.93, fill = "goldenrod") +
    geom_label(aes(x = as.numeric(ext_plot_AB[1]) + (as.numeric(ext_plot_AB[3])-as.numeric(ext_plot_AB[1]))*0.1 , 
                   y = as.numeric(ext_plot_AB[2]) + (as.numeric(ext_plot_AB[4])-as.numeric(ext_plot_AB[2]))*0.1
                   , label = "A"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_AB[1], ext_plot_AB[3]),
    ylim = c(ext_plot_AB[2], ext_plot_AB[4]),
    expand = F
  ) +
  geom_segment(data = Flight_AB, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_AB, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_AB, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map()+
  labs(x="Longitude",
       y="Latitude")+
  scale_x_continuous(breaks = -3.7) +
  scale_y_continuous(breaks = 47.8) +
  # Move y-axis to the right
  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 7.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -7, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    legend.box = "vertical",  # Stack legends vertically
    legend.box.just = "left",  # Align the legends to the left
    legend.spacing.y = unit(0.0, 'mm'),  # Adjust the spacing between items
    legend.key = element_rect(fill = "white", color = "black"),  # Ensure clear legend keys
    legend.box.margin = margin(5, 5, 5, 5),
    panel.grid.major = ggplot2::element_line(linetype = "dotted", 
      colour = alpha("grey30",0.5), linewidth = 0.25)
    )  # Adjust position if needed  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_AB.png",plot_AB,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_AB.png",plot_AB,width= 10, height=10, dpi = 200)  

In [ ]:
It_Etel <- "Data/shp/Intertidal_sites/mask_intertidal_Etel.shp" %>% 
  read_sf()

Land_Etel <- "Data/shp/Intertidal_sites/mask_Land_Etel.shp" %>% 
  read_sf()



ext_plot_Etel <- st_bbox(create_square_sf(10000, c(-3.186821, 47.683477)))


Flight_Etel<-data.frame(
  Name=c(
  "Berringue",
  "Lorois"),
  Long=c(-3.194274,
         -3.205714) ,
  Lat=c(47.698978,
        47.675159)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Berringue" ~ lon - 1000,
                               Name == "Lorois" ~ lon + 2000),
         lat_label = case_when(Name ==  "Berringue" ~ lat + 1000,
                               Name == "Lorois" ~ lat - 1000))


(plot_Etel <- ggplot() +
  geom_sf(data = Land_Etel, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_Etel, linewidth = 0.05, alpha = 0.93, fill = "goldenrod") +
    geom_label(aes(x = as.numeric(ext_plot_Etel[1]) + (as.numeric(ext_plot_Etel[3])-as.numeric(ext_plot_Etel[1]))*0.1 , 
                   y = as.numeric(ext_plot_Etel[2]) + (as.numeric(ext_plot_Etel[4])-as.numeric(ext_plot_Etel[2]))*0.1
                   , label = "B"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_Etel[1], ext_plot_Etel[3]),
    ylim = c(ext_plot_Etel[2], ext_plot_Etel[4]),
    # ndiscr = 1,
    expand = FALSE
  ) +
  scale_x_continuous(breaks = -3.2) +
  scale_y_continuous(breaks = 47.68) +
  geom_segment(data = Flight_Etel, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_Etel, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_Etel, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map() +
  labs(x = "Longitude", y = "Latitude")+
  # scale_x_continuous(breaks = seq(floor(ext_plot_Etel[1]), ceiling(ext_plot_Etel[3]), by = 500)) +
  # scale_y_continuous(breaks = seq(floor(ext_plot_Etel[2]), ceiling(ext_plot_Etel[4]), by = 500)) +

  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 7.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -7, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    legend.box = "vertical",
    legend.box.just = "left",
    legend.spacing.y = unit(0.0, 'mm'),
    legend.key = element_rect(fill = "white", color = "black"),
    legend.box.margin = margin(5, 5, 5, 5),
    panel.grid.major = element_line(
      linetype = "dotted",
      colour = alpha("grey30", 0.5),
      linewidth = 0.25
    )
  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_Etel.png",plot_AB,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Etel.png",plot_AB,width= 10, height=10, dpi = 200)  

In [ ]:
(final_map <- ((p1 | plot_AB) / (plot_Etel | plot_Saja))  + 
  plot_layout(guides = "collect")& 
    # theme_void()&
  theme(legend.position = "bottom",
        plot.margin = unit(c(0, 0, 0, 0), "cm"),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    axis.text.x = element_text(size = 20, vjust = 6, hjust = 0.8),
    axis.text.y = element_text(size = 20, vjust = -5, hjust = 0, angle = 90))
)

ggsave("Paper/Figures/High_res/Figure1/Map_site.png",final_map,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_site.png",final_map,width= 10, height=10, dpi = 200)

In [ ]:
knitr::include_graphics("Figures/Low_res/Figure1/Map_site.png")

## 2.2 Drone flights

In [ ]:
df <- data.frame(Country = c("France", "France", "France", "France", "Spain", "Spain"),
                 Site = c("Aven & Belon","Aven & Belon","Ria d'Etel","Ria d'Etel", "Saja Estuary","Saja Estuary"),
                 Flight  = c("Notre-Dame De Tremor","Pont du Guilly","Berringue","Lorois","Marisma de Cortiguera","Marisma de Cudón"),
                 Date = c("2024-04-11","2024-04-11","2023-10-14","2023-10-15","2024-06-25","2024-06-25"), 
                 Area = c(266582,213064,25486+127818,45293,204133,83757)) %>% 
  mutate(Area = round(Area * 0.0001,1))



brdr1 <- fp_border_default(color = "black", width = 0.5)
brdr2 <- fp_border_default(color = "grey", width = 1)
brdr3 <- fp_border_default(color = "grey40", width = 1)

 i = 16.5 # width of the side borders in the word_document output (in centimeters)
 w = i*0.3937 # width of the side borders in the word_document output (in inches)
 
flx1 <- flextable(df)  %>%
  flextable::width(width = (w/(ncol(df)))) %>% 
  merge_v(j = 1) %>%
  merge_v(j = 2) %>%
    border_remove() %>%
    hline_top(border=brdr1) %>%
    hline(i=6, border=brdr1) %>% 
    hline(i=4, border=brdr1) %>% 
    hline(i=2, border=brdr1) %>% 
    hline(i=1, border=brdr2)%>% 
    hline(i=3, border=brdr2)%>% 
    hline(i=5, border=brdr2) %>% 
    flextable::align(align = "center",part = "all") %>%
    flextable::align(align = "center",part = "header") %>%
  # set_caption(caption = "Table 1 List of drone Flight, summarising the date, the altitude and the purpose of each flight.") %>%
  set_table_properties(layout = "autofit") 

save_as_image(flx1, "Paper/Figures/High_res/table_flights.png", res = 400)

### 2.2.1 Multispectral data

At each location (<a href="#tbl-flights" class="quarto-xref">Table 1</a>), reflectance images with a resolution of 1.2 million pixels were captured using a DJI Matrice 300 quadcopter drone equipped with a Micasense RedEdge Dual MX multispectral camera. The camera recorded data across ten spectral bands, spanning from blue to near-infrared (NIR) wavelengths (444, 475, 531, 560, 650, 668, 705, 717, 740, and 840 nm) (). To ensure consistent lighting conditions, the drone’s flight trajectory was aligned to maintain a solar azimuth angle of 90 degrees. Image acquisition was carried out with an overlap of 70% between side-by-side images and 80% between successive images along the flight path. A downwelling light sensor (DLS2) was used to measure real-time irradiance, enabling the correction of reflectance values for variations in light intensity caused by cloud cover during the flight. The raw image data were subsequently calibrated to reflectance using a calibration panel with ~50% reflectivity, provided by the camera’s manufacturer. Images were processed using structure-from-motion photogrammetry software ([Agisoft, 2019](#ref-agisoft)) to generate multispectral orthomosaics for each flight. The orthomosaicking workflow was consistent across all flights. Initially, key tie points were identified within each image and across overlapping images to create a sparse point cloud. This point cloud was refined by removing noisy points using a reprojection accuracy metric. Subsequently, a dense point cloud was generated using a structure-from-motion algorithm. A digital surface model (DSM) was then created through surface interpolation of the dense point cloud, which served as the basis for reconstructing the multispectral ortho-image ([Nebel et al., 2020](#ref-nebel2020review)). The resolution of the multispectral orthomosaic obtained were 8 cm per pixel.

### 2.2.2 LiDAR data

Using the Matrice 300 Series Dual Gimbal Connector, a DJI Zenmuse L1 LiDAR and RGB sensor was mounted on the drone alongside a multispectral camera. This setup enabled the simultaneous capture of LiDAR point clouds, high-resolution RGB images, and multispectral images collected by the MicaSense RedEdge Dual MX during the same flight. The same processing workflow was applied to process LiDAR RGB images, resulting in orthomosaic with a resolution of 2.5 cm per pixel. Since the mapping focused solely on flat surfaces without dense vegetation, the LiDAR measured only a single return. Operating in repetitive scanning mode with a sampling rate of 240 kHz, the system achieved a point density of 350 points per square meter. The LiDAR point cloud was extracted and converted into LAS format using DJI Terra software. The LAS point cloud was then imported into Agisoft Metashape ([Agisoft, 2019](#ref-agisoft)) to generate a DEM with a resolution of 2.5 cm.

In [ ]:
knitr::include_graphics("Figures/High_res/table_flights.png")

# 3. Results

# 4. Discussion

# 5. Conclusion

# 6. References

Abreu, M.H., Pereira, R., Buschmann, A., Sousa-Pinto, I., Yarish, C., 2011. Nitrogen uptake responses of gracilaria vermiculophylla (ohmi) papenfuss under combined and single addition of nitrate and ammonium. Journal of Experimental Marine Biology and Ecology 407, 190–199.

Agisoft, 2019. [Agisoft metashape](https://www.agisoft.com/).

Barbier, E.B., Hacker, S.D., Kennedy, C., Koch, E.W., Stier, A.C., Silliman, B.R., 2011. The value of estuarine and coastal ecosystem services. Ecological monographs 81, 169–193.

Blanchet, H., Gouillieux, B., Alizier, S., others, 2014. Multiscale patterns in the diversity and organization of benthic intertidal fauna among french atlantic estuaries. Journal of Sea Research 90, 95–110. <https://doi.org/10.1016/j.seares.2014.02.014>

Capdevila, P., Hereu, B., Salguero-Gómez, R., Rovira, G. la, Medrano, A., Cebrian, E., Garrabou, J., Kersting, D.K., Linares, C., 2019. Warming impacts on early life stages increase the vulnerability and delay the population recovery of a long-lived habitat-forming macroalga. Journal of Ecology 107, 1129–1140.

Castaing, P., Guilcher, A., 1995. Morphosedimentary evolution of ria-type estuaries. Earth Surface Processes and Landforms 20, 361–376. <https://doi.org/10.1002/esp.3290200408>

Cianfaglione, K., 2021. Plant landscape and models of french atlantic estuarine systems: Extended summary of the doctoral thesis. Transylvanian Review of Systematical and Ecological Research 23, 15–36. <https://doi.org/10.2478/trser-2021-0002>

Davies, B.F.R., Oiry, S., Rosa, P., Zoffoli, M.L., Sousa, A.I., Thomas, O.R., Smale, D.A., Austen, M.C., Biermann, L., Attrill, M.J., others, 2024b. A sentinel watching over inter-tidal seagrass phenology across western europe and north africa. Communications Earth & Environment 5, 382.

Davies, B.F.R., Oiry, S., Rosa, P., Zoffoli, M.L., Sousa, A.I., Thomas, O.R., Smale, D.A., Austen, M.C., Biermann, L., Attrill, M.J., others, 2024a. Intertidal seagrass extent from sentinel-2 time-series show distinct trajectories in western europe. Remote Sensing of Environment 312, 114340.

De Valck, J., Jarvis, D., Coggan, A., Schirru, E., Pert, P., Graham, V., Newlands, M., 2023. Valuing ecosystem services in complex coastal settings: An extended ecosystem accounting framework for improved decision-making. Marine Policy 155, 105761.

Gentric, C., Sauleau, P., 2024. Bacterial load mitigation of the shellfish magallana gigas by the marine sponge hymeniacidon perlevis (montagu 1818). Regional Studies in Marine Science 75, 103564. <https://doi.org/10.1016/j.rsma.2024.103564>

Ginneken, V. van, Vries, E. de, others, 2018. The global dispersal of the non-endemic invasive red alga gracilaria vermiculophylla in the ecosystems of the euro-asia coastal waters including the wadden sea unesco world heritage coastal area: Awful or awesome? Oceanography & Fisheries Open Access Journal 8, 4–26.

Hall-Spencer, J.M., Harvey, B.P., 2019. Ocean acidification impacts on coastal ecosystem services due to habitat degradation. Emerging Topics in Life Sciences 3, 197–206.

He, Q., Silliman, B.R., 2019. Climate change, human impacts, and coastal ecosystems in the anthropocene. Current Biology 29, R1021–R1035.

Jeanneau, L., Jardé, E., Louis, J., Pannard, A., Liotaud, M., Andrieux-Loyer, F., Gruau, G., Caradec, F., Rabiller, E., Lebris, N., Laverman, A., 2023. How the origin of sedimentary organic matter impacts the benthic nutrient fluxes in shallow coastal mudflats. Comptes Rendus Géoscience 355, 237–258. <https://doi.org/10.5802/crgeos.228>

Krueger-Hadfield, S., 2018. Everywhere you look, everywhere you go, there’s an estuary invaded by the red seaweed gracilaria vermiculophylla (ohmi) papenfuss, 1967. BioInvasions Records 7.

Krueger-Hadfield, S.A., Kollars, N.M., Strand, A.E., Byers, J.E., Shainker, S.J., Terada, R., Greig, T.W., Hammann, M., Murray, D.C., Weinberger, F., others, 2017. Genetic identification of source and likely vector of a widespread marine invader. Ecology and evolution 7, 4432–4447.

Liquete, C., Piroddi, C., Drakou, E.G., Gurney, L., Katsanevakis, S., Charef, A., Egoh, B., 2013. Current status and future prospects for the assessment of marine and coastal ecosystem services: A systematic review. PloS one 8, e67737.

Liu, C., Zou, D., Liu, Z., Ye, C., 2020. Ocean warming alters the responses to eutrophication in a commercially farmed seaweed, gracilariopsis lemaneiformis. Hydrobiologia 847, 879–893.

Lu, Y., Yuan, J., Lu, X., Su, C., Zhang, Y., Wang, C., Cao, X., Li, Q., Su, J., Ittekkot, V., others, 2018. Major threats of pollution and climate change to global coastal ecosystems and enhanced management for sustainability. Environmental Pollution 239, 670–680.

Mehvar, S., Filatova, T., Dastgheib, A., De Ruyter van Steveninck, E., Ranasinghe, R., 2018. Quantifying economic value of coastal ecosystem services: A review. Journal of marine science and engineering 6, 5.

Michel, G., Le Bot, S., Lesourd, S., Lafite, R., 2021. Morpho-sedimentological and dynamic patterns in a ria type estuary: The belon estuary (south brittany, france). Journal of Maps 17, 389–400. <https://doi.org/10.1080/17445647.2021.1925170>

Mukherjee, S., Ghosh, K.K., Chanda, A., 2023. Coastal pollution—an overview. Environmental Oceanography and Coastal Dynamics: Current Scenario and Future Trends 99–107.

Nebel, S., Beege, M., Schneider, S., Rey, G.D., 2020. A review of photogrammetry and photorealistic 3D models in education from a psychological perspective, in: Frontiers in Education. Frontiers Media SA, p. 144.

Nyberg, C.D., Thomsen, M.S., Wallentinus, I., 2009. Flora and fauna associated with the introduced red alga gracilaria vermiculophylla. European Journal of Phycology 44, 395–403.

Ortega, T., Ponce, R., Forja, J., Gómez-Parra, A., 2005. Fluxes of dissolved inorganic carbon in three estuarine systems of the cantabrian sea (north of spain). Journal of Marine Systems 53, 125–142.

Portas, A., Carriot, N., Ortalo-Magné, A., Damblans, G., Thiébaut, M., Culioli, G., Quillien, N., Briand, J.-F., 2023. Impact of hydrodynamics on community structure and metabolic production of marine biofouling formed in a highly energetic estuary. Marine Environmental Research 192, 106241. <https://doi.org/10.1016/j.marenvres.2023.106241>

Ramus, A.P., Silliman, B.R., Thomsen, M.S., Long, Z.T., 2017. An invasive foundation species enhances multifunctionality in a coastal ecosystem. Proceedings of the national academy of sciences 114, 8580–8585.

Román, A., Oiry, S., Davies, B.F., Rosa, P., Gernez, P., Tovar-Sánchez, A., Navarro, G., Méléder, V., Barillé, L., 2024. Mapping intertidal microphytobenthic biomass with very high-resolution remote sensing imagery in an estuarine system. Science of The Total Environment 177025.

Román, A., Prasyad, H., Oiry, S., Davies, B.F., Brunier, G., Barillé, L., 2023. Mapping intertidal oyster farms using unmanned aerial vehicles (UAV) high-resolution multispectral data. Estuarine, Coastal and Shelf Science 291, 108432.

Román, A., Tovar-Sánchez, A., Olivé, I., Navarro, G., 2021. Using a UAV-mounted multispectral camera for the monitoring of marine macrophytes. Frontiers in Marine Science 8, 722698.

Romero, M., Andrés, A., Alonso, R., Viguri, J., Rincón, J.M., 2008. Sintering behaviour of ceramic bodies from contaminated marine sediments. Ceramics International 34, 1917–1924.

Rueness, J., 2005. Life history and molecular sequences of gracilaria vermiculophylla (gracilariales, rhodophyta), a new introduction to european waters. Phycologia 44, 120–128.

Seitz, R.D., Wennhage, H., Bergström, U., Lipcius, R.N., Ysebaert, T., 2014. Ecological value of coastal habitats for commercially and ecologically important species. ICES Journal of Marine Science 71, 648–665.

Small, C., Nicholls, R.J., 2003. A global analysis of human settlement in coastal zones. Journal of coastal research 584–599.

Sotka, E.E., Baumgardner, A.W., Bippus, P.M., Destombe, C., Duermit, E.A., Endo, H., Flanagan, B.A., Kamiya, M., Lees, L.E., Murren, C.J., others, 2018. Combining niche shift and population genetic analyses predicts rapid phenotypic evolution during invasion. Evolutionary Applications 11, 781–793.

Tankoua, O.F., Buffet, P.-E., Amiard, J.-C., Amiard-Triquet, C., Mouneyrac, C., Berthet, B., 2011. Potential influence of confounding factors (size, salinity) on biomarkers in the sentinel species scrobicularia plana used in programmes monitoring estuarine quality. Environmental Science and Pollution Research 18, 1253–1263. <https://doi.org/10.1007/s11356-011-0479-3>

Thomsen, M.S., Stæhr, P.A., Nejrup, L., Schiel, D.R., 2013. Effects of the invasive macroalgae gracilaria vermiculophylla on two co-occurring foundation species and associated invertebrates. Aquatic Invasions 8, 133–145.

Unsworth, R.K., Cullen-Unsworth, L.C., Jones, B.L., Lilley, R.J., 2022. The planetary role of seagrass conservation. Science 377, 609–613.

Van Katwijk, M., 2003. Reintroduction of eelgrass (zostera marina l.) in the dutch wadden sea: A research overview and management vision, in: Challenges to the Wadden Sea Area. In: Proceedings of the 10th International Scientific Wadden Sea Symposium, Groningen, the Netherlands. pp. 173–195.

Watanabe, Y., Kawamura, T., Yamashita, Y., 2018. Introduction: The coastal ecosystem complex as a unit of structure and function of biological productivity in coastal areas. Fisheries science 84, 149–152.

Weinberger, F., Buchholz, B., Karez, R., Wahl, M., 2008. The invasive red alga gracilaria vermiculophylla in the baltic sea: Adaptation to brackish water may compensate for light limitation. Aquatic Biology 3, 251–264.

Zoffoli, M.L., Gernez, P., Godet, L., Peters, S., Oiry, S., Barillé, L., 2021. Decadal increase in the ecological status of a north-atlantic intertidal seagrass meadow observed with multi-mission satellite time-series. Ecological Indicators 130, 108033.